In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

from tqdm.notebook import tqdm, trange

%matplotlib inline

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
config = dict(
    n_epochs=5,
    batch_size=128,
    learning_rate=1e-3,
    weight_decay=0.01,
    target_loss=0.05
)

In [16]:
class PairsDataset(torch.utils.data.Dataset):

    def __init__(self, source_file):
        self.__data = pd.read_csv(source_file).dropna().sort_values(by=["time", "rs_id", "id"]).reset_index(drop=True).drop(columns=["load_time"])
        self.__data = self.__data[[
            "time", "id",
            "x", "y", "z",
            "x_err", "y_err", "z_err"
        ]]

    def __len__(self):
        return len(self.__data)
        
    def __getitem__(self, _):
        idx1 = np.random.randint(len(self.__data))

        t = self.__data.loc[idx1, "time"]
        t_idxs = self.__data[self.__data["time"] == self.__data.loc[idx1, "time"]].index

        idx2 = -1
        while idx2 == idx1 or idx2 == -1:
            idx2 = np.random.choice(t_idxs)

        x1 = self.__data.iloc[idx1]
        x2 = self.__data.iloc[idx2]
        y = x1["id"] == x2["id"]
        
        x1 = torch.tensor(x1.drop(["id", "time"]), dtype=torch.float32)
        x2 = torch.tensor(x2.drop(["id", "time"]), dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)

        return x1, x2, y

In [17]:
class SiameseNetwork(torch.nn.Module):
    
    def __init__(self, input_size):
        super(SiameseNetwork, self).__init__()

        self.layers_head = torch.nn.Sequential(
            torch.nn.Linear(input_size, input_size),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(input_size, input_size),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(input_size, input_size)
        )

        self.layers_tail = torch.nn.Sequential(
            torch.nn.Linear(2 * input_size, 2 * input_size),
            torch.nn.LeakyReLU(),
            torch.nn.Linear(2 * input_size, 1),
            torch.nn.Sigmoid()
        )

        self.layers_head.apply(self.init_weights)
        self.layers_tail.apply(self.init_weights)

    def init_weights(self, m):
        if isinstance(m, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)
    
    def forward(self, x1, x2):
        out1 = self.layers_head(x1)
        out2 = self.layers_head(x2)

        out = torch.cat((out1, out2), 1)
        
        out = self.layers_tail(out)

        return out

In [18]:
train_dataset = PairsDataset(source_file='data/train/2ao_2rs_2902231044.csv')

In [19]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=config["batch_size"], shuffle=True)

In [20]:
model = SiameseNetwork(input_size=train_dataset[0][0].shape[0])
optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])

criterion = torch.nn.BCELoss()

In [21]:
losses = []
lrs = []
model.train()

epoch_progress = trange(config["n_epochs"])
for epoch in epoch_progress:
    epoch_progress.set_description(f'Epoch {epoch + 1}')

    batch_progress = tqdm(train_dataloader)
    for x1, x2, y in batch_progress:
        batch_progress.set_description(f'Epoch {epoch + 1}, batches')
        output = model(x1, x2).squeeze()
        loss = criterion(output, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
        lrs.append(optimizer.param_groups[0]['lr'])
        losses.append(loss.item())

        batch_progress.set_postfix({'loss': loss.item()})

    if losses[-1] < config["target_loss"]:
        break

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/1875 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12, 6))

# Loss plot
axis[0].set_title("Loss")
axis[0].set_xlabel("N_epoch")
axis[0].set_ylabel("Loss")
axis[0].semilogy()
axis[0].grid()
axis[0].plot(losses)

# Learning rate plot
axis[1].set_title("Learning rate")
axis[1].set_xlabel("N_epoch")
axis[1].set_ylabel("Learning rate")
axis[1].grid()
axis[1].plot(lrs)

In [ ]:
# torch.save(model.state_dict(), "model-25022333.pt")

In [ ]:
with torch.inference_mode():
    output = model(x1, x2).squeeze()
    loss = criterion(output, y)
    print(f"Loss = {loss.item()}")
    print(f"Actual: {y}")
    print(f"Predict: {output}")

In [ ]:
test_dataset = PairsDataset(source_file='data/test/2ao_2rs_0103175435.csv')

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

In [ ]:
for x1, x2, y in test_loader:
    with torch.inference_mode():
        output = model(x1, x2).squeeze()
        loss = criterion(output, y)
        print(f"Loss = {loss.item()}")
        print(f"Actual: {y}")
        print(f"Predict: {output}")

In [ ]:
assert 1 == 0

In [ ]:
velocities = [
    0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 0.8, 1.0, 1.5, 2.0, 2.5, 2.7, 3.0
]

for v in velocities:
    df_ = pd.read_csv(f'data/test/1ao_2rs_axis-x_track-{v}t.csv').dropna()
    x_test_, y_test_ = dataframe_to_tensors(df_, targets=['is_identical'])
    with torch.no_grad():
        print(f'Test loss on new system (velocity - {v}) : {criterion(model(x_test_), y_test_)}')
        print(model(x_test_))

In [ ]:
df_ = pd.read_csv(f'data/test/2ao_2rs_axis-x.csv').dropna()
x_test_, y_test_ = dataframe_to_tensors(df_, targets=['is_identical'])
with torch.no_grad():
    print(f'Test loss on new system : {criterion(model(x_test_), y_test_)}')
    print(model(x_test_)[:10])
    print(y_test_[:10])

In [ ]:
assert 1 == 0

In [ ]:
def log_crapped(x):
    return np.max([np.log(x), -100.0])

def bce_loss(x, y):
    return - (y * log_crapped(x) + (1.0 - y) * log_crapped(1.0 - x))

def mse_loss(x, y):
    return (x - y)**2

def mae_loss(x, y):
    return np.abs(x - y)

targets = [0.0, 1.0]
predictions = [0, 0.01, 0.5, 0.99, 1.0]

for xx in predictions:
    for yy in targets:
        print(f'prediction - {xx}, target - {yy}:')
        print(f'    bce_loss - {bce_loss(xx, yy)}')
        print(f'    mse_loss - {mse_loss(xx, yy)}')
        print(f'    mae_loss - {mae_loss(xx, yy)}')